In [ ]:
# URL to the data
url = 'https://raw.githubusercontent.com/NiceVincent/NLP_group5/master/data/large/Gift_Cards.json'

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import glob
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
from xml.etree import ElementTree
import warnings
import shutil
import random

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [ ]:
annotations_directory = '../input/face-mask-detection/annotations'
images_directory = '../input/face-mask-detection/images'

In [ ]:
# Creating a dictionary to store annotation information
information = {'xmin': [], 'ymin': [], 'xmax': [], 'ymax': [], 'label': [], 'file': [], 'width': [], 'height': []}

In [ ]:
# Parsing XML annotations
for annotation in glob.glob(annotations_directory + '/*.xml'):
    tree = ElementTree.parse(annotation)
    for element in tree.iter():
        if 'size' in element.tag:
            for attribute in list(element):
                if 'width' in attribute.tag:
                    width = int(round(float(attribute.text)))
                if 'height' in attribute.tag:
                    height = int(round(float(attribute.text)))

        if 'object' in element.tag:
            for attribute in list(element):
                if 'name' in attribute.tag:
                    name = attribute.text
                    information['label'] += [name]
                    information['width'] += [width]
                    information['height'] += [height]
                    information['file'] += [annotation.split('/')[-1][0:-4]]

                if 'bndbox' in attribute.tag:
                    for dimension in list(attribute):
                        if 'xmin' in dimension.tag:
                            xmin = int(round(float(dimension.text)))
                            information['xmin'] += [xmin]
                        if 'ymin' in dimension.tag:
                            ymin = int(round(float(dimension.text)))
                            information['ymin'] += [ymin]
                        if 'xmax' in dimension.tag:
                            xmax = int(round(float(dimension.text)))
                            information['xmax'] += [xmax]
                        if 'ymax' in dimension.tag:
                            ymax = int(round(float(dimension.text)))
                            information['ymax'] += [ymax]

In [ ]:
# Creating DataFrame from annotation information
annotations_info_df = pd.DataFrame(information)

# Adding Annotation and Image File Names
annotations_info_df['annotation_file'] = annotations_info_df['file'] + '.xml'
annotations_info_df['image_file'] = annotations_info_df['file'] + '.png'

# Create the 'cropped_image_file' column
annotations_info_df['cropped_image_file'] = annotations_info_df['file']

# Correcting label grammatical issue
annotations_info_df.loc[annotations_info_df['label'] == 'mask_weared_incorrect', 'label'] = 'mask_incorrectly_worn'

In [ ]:
# Function to render annotated images
def render_image(image_path):
    image = cv2.imread(image_path)
    img = image_path.split('/')[-1]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    bound_box = []

    for i in annotations_info_df[annotations_info_df['image_file'] == img].index:
        (xmin, ymin, xmax, ymax) = (annotations_info_df.loc[i].xmin, annotations_info_df.loc[i].ymin,
                                     annotations_info_df.loc[i].xmax, annotations_info_df.loc[i].ymax)
        bound_box.append((xmin, ymin, xmax, ymax))

        if annotations_info_df.loc[i].label == 'with_mask':
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 200, 0), 2)
            cv2.putText(image, org=(xmin - 8, ymin - 8), text="Mask",
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 200, 0))
        elif annotations_info_df.loc[i].label == 'mask_incorrectly_worn':
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 255, 0), 2)
            cv2.putText(image, org=(xmin - 8, ymin - 3), text='Incorrect',
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255, 255, 0))
        else:
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (200, 0, 0), 2)
            cv2.putText(image, org=(xmin - 8, ymin - 3), text='No mask',
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(200, 0, 0))

    plt.figure(figsize=(5, 5))
    plt.imshow(image)
    plt.show()

    return bound_box, image

In [ ]:
# Processing images with two annotations
images_with_2 = []
for _ in annotations_info_df['image_file'].value_counts().index:
    if annotations_info_df[annotations_info_df['image_file'] == _].shape[0] == 2:
        images_with_2.append(_)

In [ ]:
# Displaying images with two annotations
for i in images_with_2[:3]:
    render_image(os.path.join(images_directory, i))

In [ ]:
# Getting bounding boxes and rendering cropped images
bound_box, image = render_image(os.path.join(images_directory, i))
for i in bound_box:
    cropped = image[i[1]:i[3], i[0]:i[2]]
    plt.imshow(cropped)
    plt.show()

In [ ]:
# Creating directory for cropped images
directory = 'cropped_images'
parent_directory = '/kaggle/working'
path = os.path.join(parent_directory, directory)

try:
    shutil.rmtree(parent_directory)
except:
    pass

os.mkdir(path)

In [ ]:
# Cropping and saving images
print("(0) files are generated.", end="\r")
for i in range(len(annotations_info_df)):
    # Get The File Path and Read The Image
    image_filepath = '../input/face-mask-detection/images/' + annotations_info_df['image_file'].iloc[i]
    image = cv2.imread(image_filepath)

    # Set The Cropped Image File Name
    annotations_info_df['cropped_image_file'].iloc[i] = annotations_info_df['cropped_image_file'].iloc[i] + '-' + str(i) + '.png'
    cropped_image_filename = annotations_info_df['cropped_image_file'].iloc[i]

    # Get The xmin, ymin, xmax, ymax Value (Bounding Box) to Crop Image
    xmin = annotations_info_df['xmin'].iloc[i]
    ymin = annotations_info_df['ymin'].iloc[i]
    xmax = annotations_info_df['xmax'].iloc[i]
    ymax = annotations_info_df['ymax'].iloc[i]

    # Crop The Image Based on The Values Above
    cropped_image = image[ymin:ymax, xmin:xmax]

    cropped_image_directory = os.path.join('./cropped_images', cropped_image_filename)
    cv2.imwrite(cropped_image_directory, cropped_image)
    print("{} of {} files are generated.".format(i + 1, len(annotations_info_df)), end="\r")

In [ ]:
# Splitting data into train and test sets
classes = annotations_info_df['label'].unique()
labels = annotations_info_df['label']
annotations_info_df.drop(['label'], axis=1, inplace=True)
X_train, X_test, Y_train, Y_test = train_test_split(annotations_info_df, labels, test_size=0.25, stratify=labels,
                                                    random_state=42)

In [ ]:
# Displaying train and test set shapes
print(X_train.shape, X_test.shape)

In [ ]:
# Displaying train set label counts
Y_train.value_counts()

In [ ]:
# Collect image dimensions
image_width = []
image_height = []
for i in range(len(X_train)):
    cropped_image_path = './cropped_images/' + X_train['cropped_image_file'].iloc[i]
    cropped_image = cv2.imread(cropped_image_path)
    image_width.append(cropped_image.shape[0])
    image_height.append(cropped_image.shape[1])

In [ ]:
# Print statistics about image dimensions
print('IMAGE WIDTH')
print(f'Min: {min(image_width)}')
print(f'Max: {max(image_width)}')
print(f'Mean: {np.mean(image_width)}')
print(f'Median: {np.median(image_width)}')
print('IMAGE HEIGHT')
print(f'Min: {min(image_height)}')
print(f'Max: {max(image_height)}')
print(f'Mean: {np.mean(image_height)}')
print(f'Median: {np.median(image_height)}')

In [ ]:
# Set target image size
image_target_size = (60, 60)

In [ ]:
# Assign labels to training and testing sets
X_train['label'] = Y_train
X_test['label'] = Y_test

In [ ]:
# Create image data generator for training
train_image_generator = ImageDataGenerator(rescale=1. / 255.)
train_generator = train_image_generator.flow_from_dataframe(
    dataframe=X_train,
    directory='./cropped_images',
    x_col='cropped_image_file',
    y_col='label',
    subset='training',
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode='categorical',
    target_size=image_target_size
)

In [ ]:
# Create image data generator for testing
test_image_generator = ImageDataGenerator(rescale=1. / 255.)
test_generator = train_image_generator.flow_from_dataframe(
    dataframe=X_test,
    directory='./cropped_images',
    x_col='cropped_image_file',
    y_col='label',
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode='categorical',
    target_size=image_target_size
)

In [ ]:
# Define the CNN model architecture
def my_model():
    inputs = keras.Input(shape=(60, 60, 3))
    x = layers.Conv2D(32, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Conv2D(64, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Conv2D(128, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(3, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# Instantiate the model
model = my_model()
print(model.summary())

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
# Train the model
history_1 = model.fit(train_generator, epochs=10, steps_per_epoch=len(train_generator),
                      validation_data=test_generator, validation_steps=len(test_generator))

In [ ]:
# Load an example image and preprocess it for prediction
cropped_image_directory = "/kaggle/input/face-mask-detection/images"
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
cropped_image_file = os.path.join(directory, random.choice(files))

image = cv2.imread(cropped_image_file)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
bigger = cv2.resize(image, (60, 60))
bigger = bigger / 255
bigger = bigger.reshape(1, 60, 60, 3)

# Make predictions
pred_val = np.argmax(model.predict(bigger))
class_ind=train_generator.class_indices

# Map prediction index to label
y_pred = []
for i, j in class_ind.items():
    if pred_val == j:
        y_pred.append(i)

# Print predicted label
for predict in y_pred:
    print(predict)